# README

This script contains six different extraction algorithms (3 stains, 2 locations). Each algorithm employs a combination of dilation, smoothing, removing small holes, and removing small objects to obtain an epithelia mask. The sequence of these morphological functions and their parameter values were determined by trial-and-error. To tune these parameter values, I outputted an image of the epithelia mask after each function to determine the most appropriate function to use next. While the code for generating these intermediary images has been commented out, you can visualize them by simply removing the # preceding the relevant lines.

Among the six algorithms, the two created for H&E stains are the most accurate (followed by Melan-A stains; the algorithm has trouble accurately extracting epithelia from Sox-10 stains). The algorithm for H&E Sheffield stains was developed by `Prashant Kumar`, while the remaining algorithms were closely modeled on his methodology.

# To-dos

Before running this script, please complete the following steps:

1. **Set the input and output directories:** Make sure the paths for the input and output directories are correctly configured to point to the appropriate folders on your computer.
2. **Rename files to include locations:** Update the file names to include their locations (E.G., h2114189 melan_ROI_1 (Sheffield).tif). This step is essential because the extraction algorithm selection depends on both the stain type and location.
3. **Run the setup code:** Execute all code cells from the purple header through the end of the file **before executing the code cell below.**
4. **Verify the folder object:** Confirm that the `folder` object contains the correct tissue scans. Once verified, you may execute the code cell below.

# <font color = 'red'>READ THE README AND COMPLETE TO-DOs BEFORE RUNNING THE CELL BELOW</font>

In [ ]:
########################################################################################################################


unrecognized_stain_type = []
unrecognized_stain_location = []


for file in folder:
    
    if 'h&e' in file:
        if 'iverpool' in file:
            extract_liverpool_HandE(file, output_dir)
            
        elif 'heffield' in file:
            extract_sheffield_HandE(file, output_dir)
            
        else:
            print("Unrecognized stain location")
            unrecognized_stain_location.append(file)
            
        
    elif 'mela' in file:
        if 'iverpool' in file:
            extract_liverpool_MelanA(file, output_dir)
            
        elif 'heffield' in file:
            extract_sheffield_MelanA(file, output_dir)
            
        else:
            print("Unrecognized stain location")
            unrecognized_stain_location.append(file)
            

    elif 'sox10' in file:
        if 'iverpool' in file:
            extract_liverpool_Sox10(file, output_dir)
            
        elif 'heffield' in file:
            extract_sheffield_Sox10(file, output_dir)
            
        else:
            print("Unrecognized stain location")
            unrecognized_stain_location.append(file)
            
        
    elif file == '.DS_Store':
        continue
            
    else:
        print("Unrecognized stain type")
        unrecognized_stain_type.append(file)
        
        
print('Done')
print(unrecognized_stain_type)
print(unrecognized_stain_location)


########################################################################################################################

# <font color = 'purple'>SET-UP CODE: Run ALL cells below before running the cell above

In [1]:
# Libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import gc
from scipy.ndimage import gaussian_filter
from skimage import morphology
from skimage import color
from skimage import measure
from skimage.transform import rescale
import scipy.ndimage as ndimage
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
import psutil
import time
import math

### Working directory <font color = 'red'>(To-do: Modify the working directory to point to the correct folder on your computer)</font>

In [ ]:
print("Current Working Directory:", os.getcwd())
os.chdir("/Users/kevin/Downloads/Northwestern University/Data Science/STAT_390")
print("New Working Directory:", os.getcwd())

# IMPORTANT: Point to the correct input and output directories
input_dir = "Data/Test"               # all files in here will be read, expected filenames: <filename>.tif
output_dir = "Extracted Epithelium/Final Presentation/Test"

### Check the files in the working directory

In [ ]:
# Files in input directory
folder = os.listdir(input_dir)
folder

### Plotting an image

In [5]:
show_images = 1
def imshow(img, title):
    global show_images
    if show_images == 0:
        return
    
    plt.imshow(img)
    plt.tight_layout()
    plt.axis('off')
    plt.title(title)
    plt.show(block=True)
    plt.close('all')

if not os.path.isdir(input_dir):
    print("input_dir: '" + input_dir + "' directory does not exist! Exiting...")
    exit()

if not os.path.isdir(output_dir):
    os.mkdir(output_dir)
    print("output_dir: '" + output_dir + "' directory created.")

### Saving the final segmented image

In [6]:
def save_segmentation_image(img_rgb, stroma_img, epithelia_img, file, output_dir):
    fig = plt.figure(num=1, clear=True, figsize=(20, 12))
    ax_arr = fig.subplots(1, 3, sharex=True, sharey=True)
    fig.suptitle('Input - Stroma - Epithelia Segmentation Visualization', fontsize = 25)
    ax1, ax2, ax3 = ax_arr.ravel()
    ax1.imshow(img_rgb)
    ax1.set_title('img_rgb', fontsize = 20)
    ax1.set_axis_off()
    ax2.imshow(stroma_img)
    ax2.set_title('stroma_img', fontsize = 20)
    ax2.set_axis_off()
    ax3.imshow(epithelia_img)
    ax3.set_title('epithelia_img', fontsize = 20)
    ax3.set_axis_off()
    plt.tight_layout()
    filename = os.path.join(output_dir, file[:-4] + ".png")
    plt.savefig(filename)
    if show_images:
        plt.show()

# <font color = 'purple'>Extraction algorithms for each location+stain combination</font>

## Liverpool H&E

In [7]:
def extract_liverpool_HandE(f, output_dir):

    input_filepath = os.path.join(input_dir, f)
    print(input_filepath)

    # Load Image
    img_rgb = cv2.imread(input_filepath)
    img_rgb = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2RGB)
    img_ycrcb = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2YCrCb)        

    
    ### Background from lumma channel ###
    # Define binning in lumma image
    lumma_bins_n = 20
    divisor = (np.floor(255 / lumma_bins_n).astype(np.uint8))

    # Decimate lumma image into lumma_bins_n
    lumma_binned = (np.floor(img_ycrcb[:,:,0]/divisor)).astype(np.uint8)

    # Find bin with most number of pixels
    most_pixels_bin = -1;
    most_pixels = 0
    for bin_i in range(0,lumma_bins_n+1):
        n_pixels = np.count_nonzero(lumma_binned == bin_i)
        if n_pixels > most_pixels:
            most_pixels = n_pixels
            most_pixels_bin = bin_i
    print("\nlumma_binned: most_pixels_bin = " + str(most_pixels_bin) + "   most_pixels = " + str(most_pixels))

    # Find background
    background_bin = most_pixels_bin
    background = lumma_binned == background_bin
    background = morphology.remove_small_objects(background, 5000)
    background = morphology.remove_small_holes(background, 10000)
    #imshow(background, "background")

    
    ### Define binning in Red Chroma image ###
    Cr_bins_n = 100
    divisor = (np.floor(255 / Cr_bins_n).astype(np.uint8))

    # Decimate Red Chroma image into Cr_bins_n
    Cr_binned = (np.floor(img_ycrcb[:,:,2]/divisor)).astype(np.uint8)

    # Find bin with most number of pixels
    most_pixels_bin = -1;
    most_pixels = 0
    for bin_i in range(0,Cr_bins_n+1):
        n_pixels = np.count_nonzero(Cr_binned == bin_i)
        if n_pixels > most_pixels:
            most_pixels = n_pixels
            most_pixels_bin = bin_i
    print("\nCr_binned: most_pixels_bin = " + str(most_pixels_bin) + "   most_pixels = " + str(most_pixels))


    ### Find Definite Stroma ###
    stroma_bin = most_pixels_bin
    stroma = Cr_binned == stroma_bin
    stroma = stroma + (Cr_binned == stroma_bin - 1)
    stroma = stroma + (Cr_binned == stroma_bin - 2)

    # Remove background pixels from stroma
    stroma = stroma * np.invert(background)

    # Dilation
    stroma = morphology.dilation(stroma, morphology.square(3))

    # Remove small objects
    stroma = morphology.remove_small_objects(stroma, 1000)


    ### Find Epithelia ###
    epithelia_bin = stroma_bin + 2
    epithelia = Cr_binned == epithelia_bin
    epithelia = epithelia + (Cr_binned == epithelia_bin + 1)
    epithelia = epithelia + (Cr_binned == epithelia_bin + 2)
    epithelia = epithelia + (Cr_binned == epithelia_bin + 3)
    epithelia = epithelia + (Cr_binned == epithelia_bin + 4)
    #imshow(epithelia, "epithelia1")

    # Remove background pixels from epithelia
    epithelia = epithelia * np.invert(background)
    #imshow(epithelia, "epithelia2")

    # Remove stroma pixels from epithelia
    epithelia = epithelia * np.invert(stroma)
    #imshow(epithelia, "epithelia3")

    epithelia = epithelia * np.invert(img_ycrcb[:,:,1] < 120)
    #imshow(epithelia, "epithelia4")

    # Dilation
    epithelia = morphology.dilation(epithelia, morphology.square(2))
    #imshow(epithelia, "epithelia5")

    # Gaussian smoothing
    epithelia = gaussian_filter(epithelia.astype(float), sigma=2)
    epithelia = epithelia > 0.5
    #imshow(epithelia, "epithelia6")

    # Remove very small objects
    epithelia = morphology.remove_small_objects(epithelia, 100)
    #imshow(epithelia, "epithelia7")

    # Remove small holes
    epithelia = morphology.remove_small_holes(epithelia, 10000)
    #imshow(epithelia, "epithelia8")

    # Find out epithelia information at current state and remove small objects
    n_epithelia_pixels = np.count_nonzero(epithelia)
    percent_epithelia_pixels = n_epithelia_pixels / (epithelia.shape[0] * epithelia.shape[1])
    #print("percent_epithelia_pixels = " + str(percent_epithelia_pixels))

    small_obj_size_to_remove = 5000
    if percent_epithelia_pixels < 0.025:
        small_obj_size_to_remove = 500 
    elif percent_epithelia_pixels < 0.075:
        small_obj_size_to_remove = 1500
    #print("small_obj_size_to_remove = " + str(small_obj_size_to_remove))
    epithelia = morphology.remove_small_objects(epithelia, small_obj_size_to_remove)
    #imshow(epithelia, "epithelia9")

    # Dilation
    epithelia = morphology.dilation(epithelia, morphology.square(10))
    #imshow(epithelia, "epithelia10")

    # Remove small holes
    epithelia = morphology.remove_small_holes(epithelia, 20000)
    #imshow(epithelia, "epithelia11")

    # Apply the epithelia mask to image
    epithelia_img = cv2.bitwise_and(img_rgb,img_rgb,mask = (epithelia.astype(np.uint8) * 255))
    #imshow(epithelia_img, "epithelia_img")

    
    ### Expanded Stroma ###
    stroma_bin = most_pixels_bin
    stroma = Cr_binned == stroma_bin
    stroma = stroma + (Cr_binned == stroma_bin - 1)
    stroma = stroma + (Cr_binned == stroma_bin - 2)

    # Remove background pixels from stroma
    stroma = stroma * np.invert(background)

    # Remove small holes
    stroma = morphology.remove_small_holes(stroma, 10000)
    #imshow(stroma, "stroma3")

    # Dilation
    stroma = morphology.dilation(stroma, morphology.square(10))
    #imshow(stroma, "stroma4")

    # Remove small holes
    stroma = morphology.remove_small_holes(stroma, 20000)
    #imshow(stroma, "stroma5")

    # Remove background pixels from expanded stroma
    stroma = stroma * np.invert(background)

    # Remove stroma pixels from expanded stroma
    stroma = stroma * np.invert(epithelia)

    # Apply the stroma mask to image
    stroma_img = cv2.bitwise_and(img_rgb,img_rgb,mask = (stroma.astype(np.uint8) * 255))
    
    
    # Save image
    save_segmentation_image(img_rgb, stroma_img, epithelia_img, f, output_dir)

    print('---Iteration Finished---')
    print()
    print()
    print()

## Liverpool Melan-A

In [8]:
def extract_liverpool_MelanA(f, output_dir):

    input_filepath = os.path.join(input_dir, f)
    print(input_filepath)

    ### Load Image ###
    img_rgb = cv2.imread(input_filepath)
    img_rgb = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2RGB)
    img_ycrcb = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2YCrCb)


    ### Obtaining background from lumma channel ###
    # Define binning in lumma image
    lumma_bins_n = 20
    divisor = (np.floor(255 / lumma_bins_n).astype(np.uint8))

    # Decimate lumma image into lumma_bins_n
    lumma_binned = (np.floor(img_ycrcb[:,:,0]/divisor)).astype(np.uint8)

    # Find bin with most number of pixels
    most_pixels_bin = -1;
    most_pixels = 0
    for bin_i in range(0,lumma_bins_n+1):
        n_pixels = np.count_nonzero(lumma_binned == bin_i)
        if n_pixels > most_pixels:
            most_pixels = n_pixels
            most_pixels_bin = bin_i
    print("\nlumma_binned: most_pixels_bin = " + str(most_pixels_bin) + "   most_pixels = " + str(most_pixels))


    # Find background
    background_bin = most_pixels_bin
    background = lumma_binned == background_bin
    background = morphology.remove_small_objects(background, 5000)
    background = morphology.remove_small_holes(background, 10000)
    #imshow(background, "background")


    ### Red Chroma channel ###
    # We will get epithelia from Red Chroma channel
    # Define binning in Red Chroma image
    Cr_bins_n = 100
    divisor = (np.floor(255 / Cr_bins_n).astype(np.uint8))

    # Decimate Red Chroma image into Cr_bins_n
    Cr_binned = (np.floor(img_ycrcb[:,:,2]/divisor)).astype(np.uint8)

    # Find bin with most number of pixels
    most_pixels_bin = -1;
    most_pixels = 0
    for bin_i in range(0,Cr_bins_n+1):
        n_pixels = np.count_nonzero(Cr_binned == bin_i)
        if n_pixels > most_pixels:
            most_pixels = n_pixels
            most_pixels_bin = bin_i
    print("\nCr_binned: most_pixels_bin = " + str(most_pixels_bin) + "   most_pixels = " + str(most_pixels))


    ### Find Definite Stroma ###
    stroma_bin = most_pixels_bin
    stroma = Cr_binned == stroma_bin
    stroma = stroma + (Cr_binned == stroma_bin - 1)
    stroma = stroma + (Cr_binned == stroma_bin + 1)

    # Remove background pixels from stroma
    stroma = stroma * np.invert(background)

    # Dilation
    stroma = morphology.dilation(stroma, morphology.square(3))

    # Remove small objects
    stroma = morphology.remove_small_objects(stroma, 1000)


    ### Find Epithelia ###
    epithelia_bin = stroma_bin - 2     
    epithelia = Cr_binned == epithelia_bin
    epithelia = epithelia + (Cr_binned == epithelia_bin - 1)
    epithelia = epithelia + (Cr_binned == epithelia_bin - 2)
    #imshow(epithelia, "epithelia1")

    # Remove background pixels from epithelia
    epithelia = epithelia * np.invert(background)
    #imshow(epithelia, "epithelia2 -- removing background pixels")

    # Dilation
    epithelia = morphology.dilation(epithelia, morphology.square(3))
    #imshow(epithelia, "epithelia3 -- dilating by factor 3")

    # Apply gaussian smoothing to the binary mask
    epithelia = gaussian_filter(epithelia.astype(float), sigma=4)
    epithelia = epithelia > 0.25
    #imshow(epithelia, "epithelia4 -- Gaussian smoothing with sigma 4")

    # Remove small holes
    epithelia = morphology.remove_small_holes(epithelia, 2500)
    #imshow(epithelia, "epithelia5 -- removing small holes below 2500")

    # Dilation
    epithelia = morphology.dilation(epithelia, morphology.square(50)) # Determined by trial-and-error
    #imshow(epithelia, "epithelia6 -- dilating by factor 50")

    # Removing "small" objects after dilation
    epithelia = morphology.remove_small_objects(epithelia, 15000) # Determined by trial-and-error
    #imshow(epithelia, "epithelia7 -- remove small objects below 15000")

    # Remove small holes
    epithelia = morphology.remove_small_holes(epithelia, 20000)
    #imshow(epithelia, "epithelia8")

    # Apply the epithelia mask to image
    epithelia_img = cv2.bitwise_and(img_rgb,img_rgb,mask = (epithelia.astype(np.uint8) * 255))
    #imshow(epithelia_img, "epithelia_img")


    ### Expanded Stroma ###
    stroma_bin = most_pixels_bin
    stroma = Cr_binned == stroma_bin
    stroma = stroma + (Cr_binned == stroma_bin - 1)
    stroma = stroma + (Cr_binned == stroma_bin + 1)
    #imshow(stroma, "stroma1")

    # Remove background pixels from stroma
    stroma = stroma * np.invert(background)
    #imshow(stroma, "stroma2")

    # Remove small holes
    stroma = morphology.remove_small_holes(stroma, 10000)
    #imshow(stroma, "stroma3")

    # Dilation
    stroma = morphology.dilation(stroma, morphology.square(10))
    #imshow(stroma, "stroma4")

    # Remove small holes
    stroma = morphology.remove_small_holes(stroma, 20000)
    #imshow(stroma, "stroma5")

    # Remove background pixels from expanded stroma
    stroma = stroma * np.invert(background)
    #imshow(stroma, "stroma")

    # Remove stroma pixels from expanded stroma
    stroma = stroma * np.invert(epithelia)
    #imshow(stroma, "stroma")

    # Apply the stroma mask to image
    stroma_img = cv2.bitwise_and(img_rgb,img_rgb,mask = (stroma.astype(np.uint8) * 255))
    #imshow(stroma_img, "stroma_img")

    
    # Save image
    save_segmentation_image(img_rgb, stroma_img, epithelia_img, f, output_dir)
    
    print('---Iteration Finished---')
    print()
    print()
    print()

## Liverpool Sox-10

In [9]:
def extract_liverpool_Sox10(f, output_dir):

    input_filepath = os.path.join(input_dir, f)
    print(input_filepath)

    ### Load Image ###
    img_rgb = cv2.imread(input_filepath)
    img_rgb = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2RGB)
    img_ycrcb = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2YCrCb)
    #imshow(img_ycrcb, 'img_ycrcb')


    ### Obtain background from lumma channel ###

    # define binning in lumma image
    lumma_bins_n = 20
    divisor = (np.floor(255 / lumma_bins_n).astype(np.uint8))

    # decimate lumma image into lumma_bins_n
    lumma_binned = (np.floor(img_ycrcb[:,:,0]/divisor)).astype(np.uint8)

    # find bin with most number of pixels
    most_pixels_bin = -1;
    most_pixels = 0
    for bin_i in range(0,lumma_bins_n+1):
        n_pixels = np.count_nonzero(lumma_binned == bin_i)
        if n_pixels > most_pixels:
            most_pixels = n_pixels
            most_pixels_bin = bin_i
    print("\nlumma_binned: most_pixels_bin = " + str(most_pixels_bin) + "   most_pixels = " + str(most_pixels))


    ### find background ###
    background_bin = most_pixels_bin
    background = lumma_binned == background_bin
    background = morphology.remove_small_objects(background, 5000)
    background = morphology.remove_small_holes(background, 10000)
    #imshow(background, "background")


    ### Obtain epithelia from BLUE Chroma channel ###
    # define binning in BLUE Chroma image
    Cr_bins_n = 200
    divisor = (np.floor(255 / Cr_bins_n).astype(np.uint8))

    # decimate BLUE Chroma image into Cr_bins_n
    Cr_binned = (np.floor(img_ycrcb[:,:,1]/divisor)).astype(np.uint8)

    # find bin with most number of pixels
    most_pixels_bin = -1;
    most_pixels = 0
    for bin_i in range(0,Cr_bins_n+1):
        n_pixels = np.count_nonzero(Cr_binned == bin_i)
        if n_pixels > most_pixels:
            most_pixels = n_pixels
            most_pixels_bin = bin_i
    print("\nCr_binned: most_pixels_bin = " + str(most_pixels_bin) + "   most_pixels = " + str(most_pixels))


    ### Find Definite Stroma ###
    stroma_bin = most_pixels_bin
    stroma = Cr_binned == stroma_bin
    stroma = stroma + (Cr_binned == stroma_bin - 2)
    stroma = stroma + (Cr_binned == stroma_bin - 3)
    stroma = stroma + (Cr_binned == stroma_bin - 4)
    #imshow(stroma, "stroma1")

    # remove background pixels from stroma
    stroma = stroma * np.invert(background)
    #imshow(stroma, "stroma2")

    # dilation
    stroma = morphology.dilation(stroma, morphology.square(3))
    #imshow(stroma, "stroma3")

    # remove small objects
    stroma = morphology.remove_small_objects(stroma, 1000)
    #imshow(stroma, "definite stroma")


    eroded_stroma = morphology.erosion(stroma, morphology.square(7))
    #imshow(eroded_stroma, "eroded stroma")


    ### Find Epithelia ###
    epithelia_bin = stroma_bin + 1
    epithelia = Cr_binned == epithelia_bin
    epithelia = epithelia + (Cr_binned == epithelia_bin + 1)
    epithelia = epithelia + (Cr_binned == epithelia_bin + 2)
    epithelia = epithelia + (Cr_binned == epithelia_bin + 3)
    #imshow(epithelia, "epithelia1")

    # remove background pixels from epithelia
    epithelia = epithelia * np.invert(background)
    #imshow(epithelia, "epithelia2 -- removing background")

    epithelia = epithelia * np.invert(eroded_stroma)
    #imshow(epithelia, "epithelia")

    # dilation
    epithelia = morphology.dilation(epithelia, morphology.square(4))
    #imshow(epithelia, "epithelia3 -- dilating by factor 4")

    # apply Gaussian smoothing to the binary mask
    epithelia = gaussian_filter(epithelia.astype(float), sigma=3)
    epithelia = epithelia > 0.33
    #imshow(epithelia, "epithelia4 -- Gaussian smoothing by sigma 3") 

    # remove small holes
    epithelia = morphology.remove_small_holes(epithelia, 3500)
    #imshow(epithelia, "epithelia5 -- remove small holes under 3500")

    # remove small objects
    epithelia = morphology.remove_small_objects(epithelia, 250)
    #imshow(epithelia, "epithelia 6 -- remove small objects below 250")

    # dilation
    epithelia = morphology.dilation(epithelia, morphology.square(70))
    #imshow(epithelia, "epithelia7 -- dilating by factor 70")

    # Removing objects AFTER dilation
    epithelia = morphology.remove_small_objects(epithelia, 25000)
    #imshow(epithelia, "epithelia8 -- remove small objects below 20000")

    # remove small holes
    epithelia = morphology.remove_small_holes(epithelia, 20000)
    #imshow(epithelia, "epithelia9")

    # Apply the epithelia mask to image
    epithelia_img = cv2.bitwise_and(img_rgb,img_rgb,mask = (epithelia.astype(np.uint8) * 255))
    #imshow(epithelia_img, "epithelia_img")


    ### Expanded Stroma ###
    # Expanded Stroma is four bins from  max pixels Cr bin - 2 to max pixels Cr bin + 1
    stroma_bin = most_pixels_bin
    stroma = Cr_binned == stroma_bin
    stroma = stroma + (Cr_binned == stroma_bin - 2)
    stroma = stroma + (Cr_binned == stroma_bin - 3)
    stroma = stroma + (Cr_binned == stroma_bin - 4)
    #imshow(stroma, "stroma1")

    # remove background pixels from stroma
    stroma = stroma * np.invert(background)
    #imshow(stroma, "stroma2")

    # remove small holes
    stroma = morphology.remove_small_holes(stroma, 10000)
    #imshow(stroma, "stroma3")

    # dilation
    stroma = morphology.dilation(stroma, morphology.square(10))
    #imshow(stroma, "stroma4")

    # remove small holes
    stroma = morphology.remove_small_holes(stroma, 20000)
    #imshow(stroma, "stroma5")

    # remove background pixels from expanded stroma
    stroma = stroma * np.invert(background)

    # remove stroma pixels from expanded stroma
    stroma = stroma * np.invert(epithelia)
    #imshow(stroma, "stroma")

    # Apply the stroma mask to image
    stroma_img = cv2.bitwise_and(img_rgb,img_rgb,mask = (stroma.astype(np.uint8) * 255))
    #imshow(stroma_img, "stroma_img")

    
    # Save image
    save_segmentation_image(img_rgb, stroma_img, epithelia_img, f, output_dir)
    
    print('---Iteration Finished--')
    print()
    print()
    print()

## Sheffield H&E (Prashant's algorithm and set-up code)

In [10]:
# arguments must be in the specified order, matching regionprops
def image_stdev(region, intensities):
    # note the ddof arg to get the sample var if you so desire!
    return np.std(intensities[region], ddof=1)

# arguments must be in the specified order, matching regionprops
def image_var(region, intensities):
    # note the ddof arg to get the sample var if you so desire!
    return np.var(intensities[region], ddof=1)

def create_binned_representation(img_2d, label, no_of_bins = 20, bins_on_plot = 20, first_bin_on_plot = 0):
    global kShowImages
    # decimate lumma image into lumma_bins_n
    img_binned = np.clip((np.floor(img_2d.astype(np.float64) * no_of_bins / 255)).astype(np.uint8), 0, no_of_bins-1)

    # find bin with most number of pixels
    most_pixels_bin = -1;
    most_pixels = 0
    for bin_i in range(0,no_of_bins+1):
        n_pixels = np.count_nonzero(img_binned == bin_i)
        #print("bin = " + str(bin_i) + "   n_pixels = " + str(n_pixels))
        if n_pixels > most_pixels:
            most_pixels = n_pixels
            most_pixels_bin = bin_i
    print(label + "_binned: most_pixels_bin = " + str(most_pixels_bin) + "   most_pixels = " + str(most_pixels))
    return img_binned, most_pixels_bin

In [11]:
def extract_sheffield_HandE(file, output_dir):

    input_filepath = os.path.join(input_dir, file)
    print(input_filepath)

    ### Load Image ###
    img_rgb = cv2.imread(input_filepath)
    img_rgb = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2RGB)
    img_YCrCb = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2YCrCb)
    # Split the image into its channels
    img_Y, img_Cr, img_Cb = cv2.split(img_YCrCb)
    # YUV
    img_yuv_f = color.rgb2yuv(img_rgb, channel_axis=-1)

    img_u = np.clip(((img_yuv_f[:,:,1] + 1) / 2 * 255), 0, 255)
    #imshow(img_u, "img_u_f")
    img_v = np.clip(((img_yuv_f[:,:,2] + 1) / 2 * 255), 0, 255)
    #imshow(img_v, "img_v_f")
    img_u_expanded = (np.clip(((img_yuv_f[:,:,1] * 4 + 1) / 2 * 255), 0, 255)).astype(np.uint8)
    #imshow(img_u_expanded, "img_u_expanded")
    img_v_expanded = (np.clip(((img_yuv_f[:,:,2] * 4 + 1) / 2 * 255), 0, 255)).astype(np.uint8)
    #imshow(img_v_expanded, "img_v_expanded")


    ### Locate Background ###
    # Obtain background from lumma channel
    lumma_binned, lumma_most_pixels_bin = create_binned_representation(img_Y, "Lumma")
    background_bin = lumma_most_pixels_bin
    background = lumma_binned == background_bin
    background = morphology.remove_small_objects(background, 5000)
    background = morphology.remove_small_holes(background, 10000)
    #imshow(background, "background")


    ### Bin Blue Chroma channel ###
    # Obtain epithelia from Blue Chroma channel
    no_of_chroma_bins = 50
    img_u_expanded_binned_50, img_u_expanded_50_most_pixels_bin = create_binned_representation(img_u_expanded, "img_u_expanded", no_of_chroma_bins, 14, 18)

    # Bin Red Chroma channel
    img_v_expanded_binned_50, img_v_expanded_50_most_pixels_bin = create_binned_representation(img_v_expanded, "img_v_expanded", no_of_chroma_bins, 20, 24)
    del img_u_expanded, img_v_expanded


    ### Locate Definite Stroma ###
    stroma_bin = no_of_chroma_bins/2
    stroma = img_u_expanded_binned_50 == stroma_bin
    stroma = stroma + (img_v_expanded_binned_50 >= 42)
    #imshow(stroma, "stroma1")

    # remove background pixels from stroma
    stroma2 = stroma * np.invert(background)
    #imshow(stroma2, "stroma2")

    # dilation
    stroma = morphology.dilation(stroma2, morphology.square(3))
    #imshow(stroma, "stroma3")

    # remove small objects
    stroma = morphology.remove_small_objects(stroma, 1000)
    #imshow(stroma, "definite stroma")


    ### Locate Epithelia ###
    epithelia_bin = no_of_chroma_bins / 2 + 1
    epithelia = img_v_expanded_binned_50 >= epithelia_bin
    epithelia = epithelia * np.invert((img_v_expanded_binned_50 >= (epithelia_bin + 12)))
    #imshow(epithelia, "epithelia1")

    # add bins 27-32 in img_u_expanded_binned_50
    epithelia = epithelia + (img_u_expanded_binned_50 >= 27)
    epithelia = epithelia * np.invert((img_u_expanded_binned_50 >= 33))
    #imshow(epithelia, "epithelia1")

    # remove background pixels from epithelia
    epithelia = epithelia * np.invert(background)
    #imshow(epithelia, "epithelia2")

    ## and add three Bins from Lumma - background bin / 3 and two below
    epithelia = epithelia + (lumma_binned == int(background_bin / 3))
    epithelia = epithelia + (lumma_binned == int(background_bin / 3 - 1))
    epithelia = epithelia + (lumma_binned == int(background_bin / 3 - 2))
    #imshow(epithelia, "epithelia3lu")

    # remove dialated not-red bins 38-40 in img_v_expanded_binned_50
    not_red = img_v_expanded_binned_50 == 38
    not_red = not_red + (img_v_expanded_binned_50 == 39)
    not_red = not_red + (img_v_expanded_binned_50 == 40)
    not_red = morphology.dilation(not_red, morphology.square(5))
    epithelia = epithelia * np.invert(not_red)

    # remove stroma pixels from epithelia
    epithelia = epithelia * np.invert(stroma)
    #imshow(epithelia, "epithelia3")

    # remove blue ink drop region
    # mark regions with blue ink drop for removal from epithelia
    blue_ink = img_Cb > 152
    #imshow(blue_ink, "blue_ink")
    blue_ink = morphology.dilation(blue_ink, morphology.square(5))
    #imshow(blue_ink, "blue_ink")
    epithelia = epithelia * np.invert(blue_ink)
    #imshow(epithelia, "epithelia4")
    del blue_ink

    # dilation
    epithelia = morphology.dilation(epithelia, morphology.square(2))
    #imshow(epithelia, "epithelia5")

    # remove small holes
    epithelia = morphology.remove_small_holes(epithelia, 10000)
    #imshow(epithelia, "epithelia7")

    # remove very small objects
    epithelia = morphology.remove_small_objects(epithelia, 500)
    #imshow(epithelia, "epithelia6")

    # find out epithelia information at current state
    n_epithelia_pixels = np.count_nonzero(epithelia)
    percent_epithelia_pixels = n_epithelia_pixels / (epithelia.shape[0] * epithelia.shape[1])
    #print("percent_epithelia_pixels = " + str(percent_epithelia_pixels))
    small_obj_size_to_remove = 10000
    if percent_epithelia_pixels < 0.01:
        small_obj_size_to_remove = 1000
    elif percent_epithelia_pixels < 0.03:
        small_obj_size_to_remove = 2000
    #print("small_obj_size_to_remove = " + str(small_obj_size_to_remove))

    # remove small objects
    epithelia = morphology.remove_small_objects(epithelia, small_obj_size_to_remove)
    #imshow(epithelia, "epithelia8")

    # dilation
    epithelia = morphology.dilation(epithelia, morphology.square(10))
    #imshow(epithelia, "epithelia9")

    # remove small holes
    epithelia = morphology.remove_small_holes(epithelia, 20000)
    #imshow(epithelia, "epithelia10")

    # label connected areas in epithelia
    labels = measure.label(epithelia)
    img_v_dialated = morphology.dilation(img_v, morphology.square(5))
    region_props = measure.regionprops(labels, img_v_dialated, extra_properties=[image_stdev, image_var])

    # get average red chroma value in areas
    for prop in region_props:
        if prop.intensity_mean < 133 or prop.intensity_mean > 146:
            # invert areas having mean intensity lower than light red
            epithelia = epithelia * np.invert(labels == prop.label)
    #imshow(epithelia, "epithelia11")

    # find cell density in tissue from bin 4 of lumma, remove regions with low cell density from epithelia
    cells = lumma_binned <= 6
    cells = morphology.dilation(cells, morphology.square(15))
    cells = morphology.remove_small_objects(cells, 2500)

    # keep regions in epithelia that have high cell density
    # cluster connected areas in epithelia into smaller sizes
    distance = ndimage.distance_transform_edt(epithelia)
    coords = peak_local_max(distance, footprint=np.ones((100, 100)), labels=epithelia)
    mask = np.zeros(distance.shape, dtype=bool)
    mask[tuple(coords.T)] = True
    markers, _ = ndimage.label(mask)
    labels = watershed(-distance, markers, mask=epithelia)
    region_props = measure.regionprops(labels)
    #imshow(epithelia, "epithelia")

    # Apply the epithelia mask to image
    epithelia_img = cv2.bitwise_and(img_rgb,img_rgb,mask = (epithelia.astype(np.uint8) * 255))
    #imshow(epithelia_img, "epithelia_img")


    ### Expanded Stroma ###
    # remove small holes
    stroma = morphology.remove_small_holes(stroma2, 10000)
    #imshow(stroma, "stroma3")

    # dilation
    stroma = morphology.dilation(stroma, morphology.square(10))
    #imshow(stroma, "stroma4")

    # remove small holes
    stroma = morphology.remove_small_holes(stroma, 20000)
    #imshow(stroma, "stroma5")

    # remove background pixels from expanded stroma
    stroma = stroma * np.invert(background)

    # remove stroma pixels from expanded stroma
    stroma = stroma * np.invert(epithelia)
    #imshow(stroma, "stroma")

    # Apply the stroma mask to image
    stroma_img = cv2.bitwise_and(img_rgb,img_rgb,mask = (stroma.astype(np.uint8) * 255))
    #imshow(stroma_img, "stroma_img")

    
    # Save image
    save_segmentation_image(img_rgb, stroma_img, epithelia_img, file, output_dir)
    
    print('---Iteration Finished---')
    print()
    print()
    print()

## Sheffield Melan-A

In [12]:
def extract_sheffield_MelanA(f, output_dir):

    input_filepath = os.path.join(input_dir, f)
    print(input_filepath)

    ### Load Image ###
    img_rgb = cv2.imread(input_filepath)
    img_rgb = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2RGB)
    img_ycrcb = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2YCrCb)


    ### Otaining background from lumma channel ###
    # Define binning in lumma image
    lumma_bins_n = 20
    divisor = (np.floor(255 / lumma_bins_n).astype(np.uint8))

    # Decimate lumma image into lumma_bins_n
    lumma_binned = (np.floor(img_ycrcb[:,:,0]/divisor)).astype(np.uint8)

    # Find bin with most number of pixels
    most_pixels_bin = -1;
    most_pixels = 0
    for bin_i in range(0,lumma_bins_n+1):
        n_pixels = np.count_nonzero(lumma_binned == bin_i)
        if n_pixels > most_pixels:
            most_pixels = n_pixels
            most_pixels_bin = bin_i
    print("\nlumma_binned: most_pixels_bin = " + str(most_pixels_bin) + "   most_pixels = " + str(most_pixels))


    ### Find background ###
    background_bin = most_pixels_bin
    background = lumma_binned == background_bin
    background = morphology.remove_small_objects(background, 5000)
    background = morphology.remove_small_holes(background, 10000)
    #imshow(background, "background")


    ### Blue Chroma ###
    # Define binning in Blue Chroma image
    Cr_bins_n = 100
    divisor = (np.floor(255 / Cr_bins_n).astype(np.uint8))

    # Decimate Blue Chroma image into Cr_bins_n
    Cr_binned = (np.floor(img_ycrcb[:,:,1]/divisor)).astype(np.uint8)

    # Find bin with most number of pixels
    most_pixels_bin = -1;
    most_pixels = 0
    for bin_i in range(0,Cr_bins_n+1):
        n_pixels = np.count_nonzero(Cr_binned == bin_i)
        if n_pixels > most_pixels:
            most_pixels = n_pixels
            most_pixels_bin = bin_i
    print("\nCr_binned: most_pixels_bin = " + str(most_pixels_bin) + "   most_pixels = " + str(most_pixels))


    ### Find Definite Stroma ###
    stroma_bin = most_pixels_bin
    stroma = Cr_binned == stroma_bin
    stroma = stroma + (Cr_binned == stroma_bin - 1)
    stroma = stroma + (Cr_binned == stroma_bin - 2)
    #imshow(stroma, "stroma1")

    # Remove background pixels from stroma
    stroma = stroma * np.invert(background)
    #imshow(stroma, "stroma2")

    # Remove small objects
    stroma = morphology.remove_small_objects(stroma, 1000)
    #imshow(stroma, "definite stroma")
    #print('definite stroma found')

    # Created an eroded mask of the stroma
    eroded_stroma = morphology.erosion(stroma, morphology.square(7))
    #imshow(eroded_stroma, "eroded stroma test")


    ### Find Epithelia ###
    epithelia_bin = stroma_bin + 2     
    epithelia = Cr_binned == epithelia_bin
    epithelia = epithelia + (Cr_binned == epithelia_bin + 1)
    epithelia = epithelia + (Cr_binned == epithelia_bin + 2)
    epithelia = epithelia + (Cr_binned == epithelia_bin + 3)
    #imshow(epithelia, "epithelia1")

    # Remove background pixels from epithelia
    epithelia = epithelia * np.invert(background)
    #imshow(epithelia, "epithelia2 -- removing background pixels")

    # Dilation
    epithelia = morphology.dilation(epithelia, morphology.square(3))
    #imshow(epithelia, "epithelia3 -- dilating by factor 3")

    # Apply gaussian smoothing to the binary mask
    epithelia = gaussian_filter(epithelia.astype(float), sigma=3)
    epithelia = epithelia > 0.33
    #imshow(epithelia, "epithelia4 -- Gaussian smoothing with sigma 3")    

    # Remove small holes
    epithelia = morphology.remove_small_holes(epithelia, 3000)
    #imshow(epithelia, "epithelia5 -- removing small holes below 3000")

    # Remove very small objects
    epithelia = morphology.remove_small_objects(epithelia, 100)
    #imshow(epithelia, "TEST: epithelia -- remove small objects below 100")

    # Dilation
    epithelia = morphology.dilation(epithelia, morphology.square(100))
    #imshow(epithelia, "epithelia6 -- dilating by factor 100")

    # Removing "small" objects after dilation
    epithelia = morphology.remove_small_objects(epithelia, 20000)
    #imshow(epithelia, "epithelia7 -- remove small objects below 20000")

    # Remove small holes
    epithelia = morphology.remove_small_holes(epithelia, 15000)
    #imshow(epithelia, "extracted epithelia")

    # Apply the epithelia mask to image
    epithelia_img = cv2.bitwise_and(img_rgb,img_rgb,mask = (epithelia.astype(np.uint8) * 255))
    #imshow(epithelia_img, "epithelia_img")


    ### Expanded Stroma ###
    stroma_bin = most_pixels_bin
    stroma = Cr_binned == stroma_bin
    stroma = stroma + (Cr_binned == stroma_bin - 1)
    stroma = stroma + (Cr_binned == stroma_bin - 2)
    #imshow(stroma, "stroma1")

    # remove background pixels from stroma
    stroma = stroma * np.invert(background)
    #imshow(stroma, "stroma2")

    # remove small holes
    stroma = morphology.remove_small_holes(stroma, 10000)
    #imshow(stroma, "stroma3")

    # dilation
    stroma = morphology.dilation(stroma, morphology.square(10))
    #imshow(stroma, "stroma4")

    # remove small holes
    stroma = morphology.remove_small_holes(stroma, 20000)
    #imshow(stroma, "stroma5")

    # remove background pixels from expanded stroma
    stroma = stroma * np.invert(background)
    # remove stroma pixels from expanded stroma
    stroma = stroma * np.invert(epithelia)
    #imshow(stroma, "stroma")

    # Apply the stroma mask to image
    stroma_img = cv2.bitwise_and(img_rgb,img_rgb,mask = (stroma.astype(np.uint8) * 255))
    #imshow(stroma_img, "stroma_img")
    
    
    # Save image
    save_segmentation_image(img_rgb, stroma_img, epithelia_img, f, output_dir)

    print('---Iteration Finished---')
    print()
    print()
    print()

## Sheffield Sox-10

In [13]:
def extract_sheffield_Sox10(f, output_dir):

    input_filepath = os.path.join(input_dir, f)
    print(input_filepath)

    ### Load Image ###
    img_rgb = cv2.imread(input_filepath)
    img_rgb = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2RGB)
    img_ycrcb = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2YCrCb)


    ### Obtain background from lumma channel ###
    # Define binning in lumma image
    lumma_bins_n = 20
    divisor = (np.floor(255 / lumma_bins_n).astype(np.uint8))

    # Decimate lumma image into lumma_bins_n
    lumma_binned = (np.floor(img_ycrcb[:,:,0]/divisor)).astype(np.uint8)

    # Find bin with most number of pixels
    most_pixels_bin = -1;
    most_pixels = 0
    for bin_i in range(0,lumma_bins_n+1):
        n_pixels = np.count_nonzero(lumma_binned == bin_i)
        if n_pixels > most_pixels:
            most_pixels = n_pixels
            most_pixels_bin = bin_i
    print("\nlumma_binned: most_pixels_bin = " + str(most_pixels_bin) + "   most_pixels = " + str(most_pixels))

    # Find background
    background_bin = most_pixels_bin
    background = lumma_binned == background_bin
    background = morphology.remove_small_objects(background, 5000)
    background = morphology.remove_small_holes(background, 10000)
    #imshow(background, "background")


    ### Obtain epithelia from BLUE Chroma channel ###
    # Define binning in BLUE Chroma image
    Cr_bins_n = 200
    divisor = (np.floor(255 / Cr_bins_n).astype(np.uint8))

    # Decimate BLUE Chroma image into Cr_bins_n
    Cr_binned = (np.floor(img_ycrcb[:,:,1]/divisor)).astype(np.uint8)

    # Find bin with most number of pixels
    most_pixels_bin = -1;
    most_pixels = 0
    for bin_i in range(0,Cr_bins_n+1):
        n_pixels = np.count_nonzero(Cr_binned == bin_i)
        if n_pixels > most_pixels:
            most_pixels = n_pixels
            most_pixels_bin = bin_i
    print("\nCr_binned: most_pixels_bin = " + str(most_pixels_bin) + "   most_pixels = " + str(most_pixels))


    ### Find Definite Stroma ###
    stroma_bin = most_pixels_bin
    stroma = Cr_binned == stroma_bin
    stroma = stroma + (Cr_binned == stroma_bin - 2)
    stroma = stroma + (Cr_binned == stroma_bin - 3)
    stroma = stroma + (Cr_binned == stroma_bin - 4)
    #imshow(stroma, "stroma1")

    # remove background pixels from stroma
    stroma = stroma * np.invert(background)
    #imshow(stroma, "stroma2")

    # dilation
    stroma = morphology.dilation(stroma, morphology.square(3))
    #imshow(stroma, "stroma3")

    # remove small objects
    stroma = morphology.remove_small_objects(stroma, 1000)
    #imshow(stroma, "definite stroma")

    # obtain an eroded version of the stroma mask (used during epithelium extraction)
    eroded_stroma = morphology.erosion(stroma, morphology.square(7))
    #imshow(eroded_stroma, "eroded stroma test")


    ### Find Epithelia ###
    epithelia_bin = stroma_bin + 5
    epithelia = Cr_binned == epithelia_bin
    epithelia = epithelia + (Cr_binned == epithelia_bin + 1)
    epithelia = epithelia + (Cr_binned == epithelia_bin + 2)
    epithelia = epithelia + (Cr_binned == epithelia_bin + 3)
    #imshow(epithelia, "epithelia1")

    # remove background pixels from epithelia
    epithelia = epithelia * np.invert(background)
    #imshow(epithelia, "epithelia2 -- removing background")

    # remove stroma 
    epithelia = epithelia * np.invert(eroded_stroma)
    #imshow(epithelia, "epithelia")

    # dilation
    epithelia = morphology.dilation(epithelia, morphology.square(4))
    #imshow(epithelia, "epithelia3 -- dilating by factor 4")

    # apply Gaussian smoothing to the binary mask
    epithelia = gaussian_filter(epithelia.astype(float), sigma=3)
    epithelia = epithelia > 0.33  # Convert back to binary by thresholding
    #imshow(epithelia, "epithelia4 -- Gaussian smoothing by sigma 3") 

    # remove small holes
    epithelia = morphology.remove_small_holes(epithelia, 3500)
    #imshow(epithelia, "epithelia5 -- remove small holes under 3500")        

    # only need to remove SMALL objects
    epithelia = morphology.remove_small_objects(epithelia, 250)
    #imshow(epithelia, "epithelia 6 -- remove small objects below 250")

    # dilation
    epithelia = morphology.dilation(epithelia, morphology.square(100))
    #imshow(epithelia, "epithelia7 -- dilating by factor 100")

    # removing objects AFTER dilation
    epithelia = morphology.remove_small_objects(epithelia, 25000)
    #imshow(epithelia, "epithelia8 -- remove small objects below 25000")

    # remove small holes
    epithelia = morphology.remove_small_holes(epithelia, 25000)
    #imshow(epithelia, "epithelia")

    # apply the epithelia mask to image
    epithelia_img = cv2.bitwise_and(img_rgb,img_rgb,mask = (epithelia.astype(np.uint8) * 255))
    #imshow(epithelia_img, "epithelia_img")


    ### Expanded Stroma ###
    stroma_bin = most_pixels_bin
    stroma = Cr_binned == stroma_bin
    stroma = stroma + (Cr_binned == stroma_bin - 2)
    stroma = stroma + (Cr_binned == stroma_bin - 3)
    stroma = stroma + (Cr_binned == stroma_bin - 4)
    #imshow(stroma, "stroma1")

    # remove background pixels from stroma
    stroma = stroma * np.invert(background)
    #imshow(stroma, "stroma2")

    # remove small holes
    stroma = morphology.remove_small_holes(stroma, 10000)
    #imshow(stroma, "stroma3")

    # dilation
    stroma = morphology.dilation(stroma, morphology.square(10))
    #imshow(stroma, "stroma4")

    # remove small holes
    stroma = morphology.remove_small_holes(stroma, 20000)
    #imshow(stroma, "stroma5")

    # remove background pixels from expanded stroma
    stroma = stroma * np.invert(background)
    # remove stroma pixels from expanded stroma
    stroma = stroma * np.invert(epithelia)
    #imshow(stroma, "stroma")

    # apply the stroma mask to image
    stroma_img = cv2.bitwise_and(img_rgb,img_rgb,mask = (stroma.astype(np.uint8) * 255))
    #imshow(stroma_img, "stroma_img")
    
    
    # Save image
    save_segmentation_image(img_rgb, stroma_img, epithelia_img, f, output_dir)

    print('---Iteration Finished---')
    print()
    print()
    print()

**End of file**